In [1]:
pip install wbgapi pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wbgapi as wb
import pandas as pd
import altair as alt

# Chart 5

In [ ]:
# World Bank API wbgapi

# Ver todos los indicadores (son miles)
#wb.series.info()

# Buscar indicadores por palabra clave
#indicators = wb.series.list(q='account')

# Ver detalles de un indicador específico
#wb.series.metadata.get('NY.GDP.PCAP.CD')

<generator object list at 0x1181cda20>

In [3]:
# 1. Define the indicators from Global Findex and World Bank
# fx.acc.t.d: % of adults with a bank account
# fx.sav.fin.t.d: % of adults who saved at a financial institution
# NY.GDP.PCAP.CD: GDP per capita (current US$)
indicators = {
    'FX.OWN.TOTL.ZS': 'Account_Ownership',
    'NY.GNS.ICTR.ZS': 'Gross_Savings',
    'NY.GDP.PCAP.CD': 'GDP_per_Capita'
}

In [4]:
year_to_fetch = 2024

In [13]:
df = wb.data.DataFrame(indicators.keys(), time=year_to_fetch, labels=True, skipAggs=True).reset_index()

In [14]:
df

,economy,Country,FX.OWN.TOTL.ZS,NY.GDP.PCAP.CD,NY.GNS.ICTR.ZS
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213
1,ZMB,Zambia,72.702425,1187.109434,34.421066
2,YEM,"Yemen, Rep.",NaN,NaN,NaN
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263
4,VIR,Virgin Islands (U.S.),NaN,NaN,NaN
...,...,...,...,...,...
212,AND,Andorra,NaN,49303.649167,NaN
213,ASM,American Samoa,NaN,NaN,NaN
214,DZA,Algeria,35.290107,5752.990767,39.419856
215,ALB,Albania,46.069251,11377.775743,22.501612


In [27]:
df_regions = pd.DataFrame(wb.economy.list())[['id', 'value', 'region']]

In [16]:
# 3. Cleaning and renaming
df = df.rename(columns={
        'economy': 'ISO3',
        'Country': 'country_name',
        'NY.GNS.ICTR.ZS': 'gross_savings_GDP',
        'FX.OWN.TOTL.ZS': 'account_ownership',
        'NY.GDP.PCAP.CD': 'GDP_per_capita'
    })

In [29]:
df_final = pd.merge(df, df_regions[['id', 'region']], left_on='ISO3', right_on='id', how='left')

In [30]:
df

,ISO3,country_name,account_ownership,GDP_per_capita,gross_savings_GDP
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213
1,ZMB,Zambia,72.702425,1187.109434,34.421066
2,YEM,"Yemen, Rep.",NaN,NaN,NaN
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263
4,VIR,Virgin Islands (U.S.),NaN,NaN,NaN
...,...,...,...,...,...
212,AND,Andorra,NaN,49303.649167,NaN
213,ASM,American Samoa,NaN,NaN,NaN
214,DZA,Algeria,35.290107,5752.990767,39.419856
215,ALB,Albania,46.069251,11377.775743,22.501612


In [31]:
df_final

,ISO3,country_name,account_ownership,GDP_per_capita,gross_savings_GDP,id,region
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213,ZWE,SSF
1,ZMB,Zambia,72.702425,1187.109434,34.421066,ZMB,SSF
2,YEM,"Yemen, Rep.",NaN,NaN,NaN,YEM,MEA
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263,PSE,MEA
4,VIR,Virgin Islands (U.S.),NaN,NaN,NaN,VIR,LCN
...,...,...,...,...,...,...,...
212,AND,Andorra,NaN,49303.649167,NaN,AND,ECS
213,ASM,American Samoa,NaN,NaN,NaN,ASM,EAS
214,DZA,Algeria,35.290107,5752.990767,39.419856,DZA,MEA
215,ALB,Albania,46.069251,11377.775743,22.501612,ALB,ECS


In [35]:
df_final['region'].unique()

array(['SSF', 'MEA', 'LCN', 'EAS', 'ECS', 'NAC', 'SAS'], dtype=object)

In [32]:
# Limpieza: Eliminar nulos para asegurar que el Bubble Chart funcione
df_clean = df_final.dropna(subset=['gross_savings_GDP', 'account_ownership', 'GDP_per_capita'])
df_clean

,ISO3,country_name,account_ownership,GDP_per_capita,gross_savings_GDP,id,region
0,ZWE,Zimbabwe,49.521602,2497.203322,10.700213,ZWE,SSF
1,ZMB,Zambia,72.702425,1187.109434,34.421066,ZMB,SSF
3,PSE,West Bank and Gaza,39.620984,2592.305912,1.883263,PSE,MEA
5,VNM,Viet Nam,70.550753,4717.290287,36.690108,VNM,EAS
8,UZB,Uzbekistan,59.658383,3161.700106,28.297617,UZB,ECS
...,...,...,...,...,...,...,...
206,AUS,Australia,98.010378,64603.985631,22.922852,AUS,EAS
208,ARM,Armenia,71.373473,8556.214070,19.235686,ARM,ECS
209,ARG,Argentina,81.744245,13969.783660,16.697117,ARG,LCN
214,DZA,Algeria,35.290107,5752.990767,39.419856,DZA,MEA


In [36]:
# Guardar para Vega-Lite
df_clean.to_csv('ownership_savings.csv', index=False)
    
print(f"¡Éxito! Datos guardados para {len(df_clean)} países.")
print(df_clean[['country_name', 'gross_savings_GDP', 'account_ownership', 'GDP_per_capita', 'region']].head())

¡Éxito! Datos guardados para 111 países.
         country_name  gross_savings_GDP  account_ownership  GDP_per_capita  \
0            Zimbabwe          10.700213          49.521602     2497.203322   
1              Zambia          34.421066          72.702425     1187.109434   
3  West Bank and Gaza           1.883263          39.620984     2592.305912   
5            Viet Nam          36.690108          70.550753     4717.290287   
8          Uzbekistan          28.297617          59.658383     3161.700106   

  region  
0    SSF  
1    SSF  
3    MEA  
5    EAS  
8    ECS  
